In [1]:
import pandas as pd
import pickle

In [2]:
file_path = "/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/codeT5_embeddings/df_fe_embeded_by_line_base.pkl"
with open(file_path, "rb") as fh:
  df = pickle.load(fh)

/var/folders/d8/cr798cb57kzd5kz_r8g8jm380000gn/T/ipykernel_32941/3914566794.py:3: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  df = pickle.load(fh)


In [3]:
df.head()

,sample_id,severity,method,label,embeded_sequence_sum,embeded_sequence_avg
0,12150,critical,public SystemDiagnosticsDTO createSystemDi...,1,0 -7.521571 1 -7.762787 2 -10.40...,0 -0.119390 1 -0.123219 2 -0.16519...
1,10168,critical,protected void addRequiredAlertProperties(Se...,1,0 -0.616862 1 -1.671327 2 -0.71217...,0 -0.123372 1 -0.334265 2 -0.14243...
2,12783,critical,"@SuppressWarnings(""try"")\n private void...",1,0 -11.226764 1 -15.170425 2 -18.57...,0 -0.091274 1 -0.123337 2 -0.15097...
3,12754,critical,public void validateDepositDetailForUpdate...,1,0 -7.613076 1 -22.431355 2 -27.73...,0 -0.067974 1 -0.200280 2 -0.24763...
4,4277,critical,@Override\n public ListenableFuture<Voi...,1,0 -7.992052 1 -9.275466 2 -14.10...,0 -0.103793 1 -0.120461 2 -0.18320...


In [4]:
embeded_sum = df.loc[:, ['sample_id', 'severity', 'label', 'embeded_sequence_sum']]
embeded_avg = df.loc[:, ['sample_id', 'severity', 'label', 'embeded_sequence_avg']]
embeded_sum.head()

,sample_id,severity,label,embeded_sequence_sum
0,12150,critical,1,0 -7.521571 1 -7.762787 2 -10.40...
1,10168,critical,1,0 -0.616862 1 -1.671327 2 -0.71217...
2,12783,critical,1,0 -11.226764 1 -15.170425 2 -18.57...
3,12754,critical,1,0 -7.613076 1 -22.431355 2 -27.73...
4,4277,critical,1,0 -7.992052 1 -9.275466 2 -14.10...


In [5]:
print(embeded_sum["embeded_sequence_sum"].head())
print(embeded_sum["embeded_sequence_sum"].apply(type).value_counts())

0    0      -7.521571
1      -7.762787
2     -10.40...
1    0     -0.616862
1     -1.671327
2     -0.71217...
2    0     -11.226764
1     -15.170425
2     -18.57...
3    0      -7.613076
1     -22.431355
2     -27.73...
4    0      -7.992052
1      -9.275466
2     -14.10...
Name: embeded_sequence_sum, dtype: object
embeded_sequence_sum
<class 'pandas.core.series.Series'>    3027
Name: count, dtype: int64


In [6]:
def process_embedding(x):
    if x is None:
        return []  
    elif isinstance(x, pd.Series):
        return " ".join(map(str, x))
    else:
        return str(x)  

embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(process_embedding)


In [7]:
print(embeded_sum["embeded_sequence_sum"].head())

0    -7.521571159362793 -7.762787342071533 -10.4074...
1    -0.6168618202209473 -1.6713271141052246 -0.712...
2    -11.226763725280762 -15.170425415039062 -18.57...
3    -7.613076210021973 -22.431354522705078 -27.734...
4    -7.992051601409912 -9.275465965270996 -14.1064...
Name: embeded_sequence_sum, dtype: object


In [8]:
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(
    lambda x: " ".join(str(i) for i in x) if isinstance(x, pd.Series) else str(x)
)

In [9]:
import re

def parse_flat_embedding(embedding_str):
    # Ensure input is a string
    if not isinstance(embedding_str, str):
        return []
    try:
        # Extract numerical values from the string
        values = re.findall(r"[-+]?\d*\.\d+|\d+", embedding_str)
        return [float(v) for v in values]
    except Exception as e:
        print(f"Error parsing: {embedding_str} -> {e}")
        return []


In [10]:
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(parse_flat_embedding)

In [11]:
n = len(embeded_sum["embeded_sequence_sum"][0])
print(f"Embedding length: {n}")

embed_lengths = embeded_sum["embeded_sequence_sum"].apply(len)
length_counts = embed_lengths.value_counts()
most_common_length = length_counts.idxmax()

n = most_common_length
print(f"Most common sequence length: {most_common_length}")
print(f"Count of rows with this length: {length_counts[most_common_length]}")


Embedding length: 768
Most common sequence length: 768
Count of rows with this length: 2880


In [12]:
 # Function to handle the trimming or padding of the sequence
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(
    lambda x: (x[:n] if len(x) > n else x + [0] * (n - len(x))) if isinstance(x, list) else [0] * n
)

print(embeded_sum["embeded_sequence_sum"].head())

0    [-7.521571159362793, -7.762787342071533, -10.4...
1    [-0.6168618202209473, -1.6713271141052246, -0....
2    [-11.226763725280762, -15.170425415039062, -18...
3    [-7.613076210021973, -22.431354522705078, -27....
4    [-7.992051601409912, -9.275465965270996, -14.1...
Name: embeded_sequence_sum, dtype: object


In [13]:
embed_lengths = embeded_sum["embeded_sequence_sum"].apply(len)
print(embed_lengths.value_counts())


embeded_sequence_sum
768    3027
Name: count, dtype: int64


In [14]:
# Create column names for embeddings
columns = [f'emb_{i+1}' for i in range(n)]

# Create a DataFrame for embedding values
data = pd.DataFrame(embeded_sum["embeded_sequence_sum"].to_list(), columns=columns)

# Add metadata columns
data['label'] = embeded_sum['label']
data['sample_id'] = embeded_sum['sample_id']
data['severity'] = embeded_sum['severity']

# Display the result
print(data.head())


       emb_1      emb_2      emb_3      emb_4     emb_5      emb_6     emb_7  \
0  -7.521571  -7.762787 -10.407430   6.591759  7.070191  11.705015  2.640116   
1  -0.616862  -1.671327  -0.712171   1.246349  0.389413   1.486012 -0.136272   
2 -11.226764 -15.170425 -18.570446  13.167607  4.495738  24.991566  6.812870   
3  -7.613076 -22.431355 -27.734577  13.366014  4.421761  25.526434  3.585086   
4  -7.992052  -9.275466 -14.106435  11.998210  3.758430  11.559190  5.180464   

       emb_8     emb_9     emb_10  ...   emb_762    emb_763    emb_764  \
0  12.856184 -7.146132  -2.885672  ...  3.887118  11.322876 -13.805436   
1   0.673206  0.153740  -0.755592  ...  0.656833   0.333388  -0.605158   
2  10.403657  1.928948 -12.261045  ... -2.955463  29.813223 -14.873217   
3  16.604767  0.640552  -9.345515  ... -7.696403  27.382660 -11.144313   
4   7.495634 -1.092035  -8.209684  ... -0.886266  20.001431 -10.732095   

     emb_765   emb_766    emb_767    emb_768  label  sample_id  severity  

In [15]:
dataset_path = '/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/codeT5_embeddings/processed_embeddings/codeT5_base_embed_line_by_line_sum.pkl'
pd.to_pickle(data, dataset_path)

In [16]:
print(embeded_avg["embeded_sequence_avg"].head())
print(embeded_avg["embeded_sequence_avg"].apply(type).value_counts())

0    0     -0.119390
1     -0.123219
2     -0.16519...
1    0     -0.123372
1     -0.334265
2     -0.14243...
2    0     -0.091274
1     -0.123337
2     -0.15097...
3    0     -0.067974
1     -0.200280
2     -0.24763...
4    0     -0.103793
1     -0.120461
2     -0.18320...
Name: embeded_sequence_avg, dtype: object
embeded_sequence_avg
<class 'pandas.core.series.Series'>    3027
Name: count, dtype: int64


In [17]:
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(
    lambda x: " ".join(str(i) for i in x) if isinstance(x, pd.Series) else str(x)
)

In [18]:
n = len(embeded_avg["embeded_sequence_avg"][0])
print(f"Embedding length: {n}")

Embedding length: 15685


In [19]:
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(parse_flat_embedding)

In [20]:
embed_lengths = embeded_avg["embeded_sequence_avg"].apply(len)
length_counts = embed_lengths.value_counts()
most_common_length = length_counts.idxmax()

n = most_common_length
print(f"Most common sequence length: {most_common_length}")
print(f"Count of rows with this length: {length_counts[most_common_length]}")

Most common sequence length: 768
Count of rows with this length: 2065


In [21]:
# Function to handle the trimming or padding of the sequence
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(
    lambda x: (x[:n] if len(x) > n else x + [0] * (n - len(x))) if isinstance(x, list) else [0] * n
)

print(embeded_avg["embeded_sequence_avg"].head())

0    [-0.11939001828432083, -0.1232188493013382, -0...
1    [-0.12337236106395721, -0.33426541090011597, -...
2    [-0.09127449989318848, -0.1233367919921875, -0...
3    [-0.06797389686107635, -0.20027995109558105, -...
4    [-0.10379287600517273, -0.12046059966087341, -...
Name: embeded_sequence_avg, dtype: object


In [22]:
embed_lengths = embeded_avg["embeded_sequence_avg"].apply(len)
print(embed_lengths.value_counts())

embeded_sequence_avg
768    3027
Name: count, dtype: int64


In [23]:
columns = [f'emb_{i+1}' for i in range(n)]
data = pd.DataFrame(embeded_avg["embeded_sequence_avg"].to_list(), columns=columns)
data['label'] = embeded_avg['label']
data['sample_id'] = embeded_avg['sample_id']
data['severity'] = embeded_avg['severity']
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,-0.119390,-0.123219,-0.165197,0.104631,0.112225,0.185794,0.041907,0.204066,-0.113431,-0.045804,...,0.061700,0.179728,-0.219134,0.135207,-0.091973,0.392297,-0.211298,1,12150,critical
1,-0.123372,-0.334265,-0.142434,0.249270,0.077883,0.297202,-0.027254,0.134641,0.030748,-0.151118,...,0.131367,0.066678,-0.121032,0.097655,-0.000780,0.286284,-0.099197,1,10168,critical
2,-0.091274,-0.123337,-0.150979,0.107054,0.036551,0.203183,0.055389,0.084583,0.015683,-0.099683,...,-0.024028,0.242384,-0.120920,0.124597,0.049541,0.288674,-0.161974,1,12783,critical
3,-0.067974,-0.200280,-0.247630,0.119339,0.039480,0.227915,0.032010,0.148257,0.005719,-0.083442,...,-0.068718,0.244488,-0.099503,0.152847,-0.018549,0.272525,-0.134554,1,12754,critical
4,-0.103793,-0.120461,-0.183200,0.155821,0.048811,0.150119,0.067279,0.097346,-0.014182,-0.106619,...,-0.011510,0.259759,-0.139378,0.181213,0.019280,0.346813,-0.155173,1,4277,critical


In [24]:
dataset_path = '/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/codeT5_embeddings/processed_embeddings/codeT5_base_embed_line_by_line_avg.pkl'
pd.to_pickle(data, dataset_path)